<a href="https://colab.research.google.com/github/julesripoll/crypto-trading-bot/blob/jules_branch/bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tf_agents.environments import py_environment
from tf_agents.specs import array_spec
from tf_agents.trajectories import time_step as ts


class TradingEnv(py_environment.PyEnvironment):

    def __init__(self, df, tc=10, window_size=5, balance=1000,live=False):
        self._action_spec= array_spec.BoundedArraySpec(shape=(), dtype=int, minimum=0, maximum=2)
        self._observation_spec= array_spec.BoundedArraySpec(shape=(5,), dtype=float, minimum=0)
        self._episode_ended=False
        self.initial_balance=balance 
        self.current_balance=balance
        self.train_df=df
        self.train_index=window_size
        self.tc=tc
        self.window_size=window_size
        self.state= df[self.train_index-window_size:self.train_index]
        self.positioned=False
        self.buying_price=0
        if live==False:
            self.len_train=df.shape[0]-1



    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec

    def _reset(self):
        self.train_index=self.window_size
        self.current_balance=self.initial_balance
        self.state= self.train_df[self.train_index-self.window_size:self.train_index]
        print('je vais être reset')
        return ts.restart(observation=self.state)

    def reward(self, ind, buying_price):
        return self.train_df[ind]-buying_price-self.tc
    
    def _step(self, action):
        """
        step retourne 4 valeurs : 
        -observation : une observation de l'environnement, ici une séquence de prix. object
        -reward : une récompense, définie ci dessous, dépendant de l'action effectuée. float
        -done : boolean indiquant si on doit reset l'env, terminant donc l'episode. boolean
        -info : information de notre choix. dict 

        note : les actions possibles dépendent de si on est positionné ou non, et là c'est pas pris en compte.
        """
        rwd=0
        current_price=self.train_df[self.train_index]
        if action==0 : #buy
            rwd=0
            self.buying_price=current_price
            self.current_balance-=current_price
            self.positioned=True
        elif action==1: #do nothing
            rwd=0
        elif action==2: #sell
            self.positioned=False
            rwd=self.reward(self.train_index,self.buying_price)
            self.current_balance+=current_price

        self.train_index+=1
        self.state=self.train_df[self.train_index-self.window_size:self.train_index]

        if (self.train_index==self.len_train) or self.current_balance<self.initial_balance*0.9:
            self.train_index=self.window_size
            print("fini")
            step=ts.termination(reward=rwd, observation=self.state)
            print(step.step_type) #si c'est bien 2 c'est ISLAST
            return step
            """
            ts.termination retourne un TimeStep. si termination à priori ça renvoie un done = true.
            A verifier dans la cellule de validation. 
            Ici la condition d'arrêt c'est d'avoir perdu 10% de l'investissement initial ou d'avoir fini le training set
            """
        else:
            print("pas fini")
            step=ts.transition(reward=rwd, observation=self.state)
            print(step.step_type) 
            return step  #s'il ne s'agit pas de la fin d'un épisode. Step type induit par transition

    def _render(self):
        pass





In [ ]:
#from trade import TradingEnv
from tf_agents.agents import DqnAgent
import pandas as pd
from tf_agents.environments import tf_py_environment
import tensorflow as tf
from tf_agents.utils import common
from tf_agents.specs import tensor_spec
from tf_agents.networks.q_rnn_network import QRnnNetwork
from tf_agents.drivers import py_driver
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers.tf_uniform_replay_buffer import  TFUniformReplayBuffer
from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver
from tf_agents.drivers.dynamic_episode_driver import DynamicEpisodeDriver
from numpy import random
from tf_agents.metrics import tf_metrics
import numpy as np
import tf_agents.trajectories as trajectory

In [ ]:
#hyperparamètres
initial_collect_steps = 100
batch_size=32
max_length=100 #pareil que initial collect steps?

In [ ]:
#import du dataset
prices=pd.read_csv("~/Developer/Trading bots/Trading bot crypto/Data/HitBTC_ETHUSD_1h.csv", sep=",")
prices = prices.rename(columns={'Close': 'value'})
df=prices['value']
df=pd.DataFrame({'value':df.values})
df=df['value']

In [ ]:
#init de l'environnement
train_py_env=TradingEnv(df[:3000])
print(train_py_env.state)
train_env = tf_py_environment.TFPyEnvironment(train_py_env)


0    534.73
1    533.85
2    532.83
3    524.71
4    519.44
Name: value, dtype: float64


In [ ]:
time_step = train_env.reset()
rewards = []
steps = []
num_episodes = 5

for _ in range(num_episodes):
  episode_reward = 0
  episode_steps = 0
  while not time_step.is_last():
    action=random.randint(0,3)
    time_step = train_env.step(action)
    episode_steps += 1
    episode_reward += time_step.reward.numpy()
  print("ep finished")
  rewards.append(episode_reward)
  steps.append(episode_steps)
  time_step = train_env.reset()

num_steps = np.sum(steps)
avg_length = np.mean(steps)
avg_reward = np.mean(rewards)

print('num_episodes:', num_episodes, 'num_steps:', num_steps)
print('avg_length', avg_length, 'avg_reward:', avg_reward)

je vais être reset
fini
2
ep finished
je vais être reset
fini
2
ep finished
je vais être reset
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas

In [ ]:
#init du réseau
rnn_net=QRnnNetwork(
    input_tensor_spec=train_env.observation_spec(),
    action_spec=train_env.action_spec(),
    lstm_size=(40,),
    output_fc_layer_params=(20,)
    )

In [ ]:
#init de l'agent
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
global_step=tf.Variable(0)

agent=DqnAgent(
    time_step_spec=train_env.time_step_spec(),
    action_spec=train_env.action_spec(),
    q_network=rnn_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=global_step
)

In [ ]:
#some data collection policies
collect_pol=agent.collect_policy

rdm_policy = random_tf_policy.RandomTFPolicy(action_spec=train_env.action_spec(),
                                            time_step_spec=train_env.time_step_spec())

en dessous on a plusieurs essais de driver et replay buffer. Voir la doc tensorflow (notebooks dqn agent with tf-agents, et replay buffers).

In [ ]:
#ne marche pas
replay_buffer = TFUniformReplayBuffer(
    agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=1000
    )

In [ ]:
train_env.reset
num_episodes = tf_metrics.NumberOfEpisodes()
env_steps = tf_metrics.EnvironmentSteps()
observers = [replay_buffer.add_batch,num_episodes, env_steps]
replay_buffer.clear()
driver = DynamicEpisodeDriver(
    train_env, collect_pol, observers,num_episodes=10)

# Initial driver.run will reset the environment and initialize the policy.
final_time_step, policy_state = driver.run()

je vais être reset
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1


2022-05-07 21:05:24.326039: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1
pas fini
1

In [ ]:
# Read the replay buffer as a Dataset,
# read batches of 4 elements, each with 2 timesteps:
dataset = replay_buffer.as_dataset(
    sample_batch_size=4,
    num_steps=2)

iterator = iter(dataset)

num_train_steps = 10

for _ in range(num_train_steps):
  trajectories, _ = next(iterator)
  loss = agent.train(experience=trajectories)

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


In [ ]:
#init de l'environnement
train_py_env=TradingEnv(df[3000:])
eval_env = tf_py_environment.TFPyEnvironment(train_py_env)

In [ ]:
try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

# Reset the train step.
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
returns = [avg_return]

# Reset the environment.
time_step = train_py_env.reset()

# Create a driver to collect experience.
collect_driver = py_driver.PyDriver(
    env,
    py_tf_eager_policy.PyTFEagerPolicy(
      agent.collect_policy, use_tf_function=True),
    [rb_observer],
    max_steps=collect_steps_per_iteration)

for _ in range(num_iterations):

  # Collect a few steps and save to the replay buffer.
  time_step, _ = collect_driver.run(time_step)

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = agent.train(experience).loss

  step = agent.train_step_counter.numpy()

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
    print('step = {0}: Average Return = {1}'.format(step, avg_return))
    returns.append(avg_return)

le problème est au niveau du calcul de la loss. Je crois que le 40 implique que je prends 40 trajectoires. Voir comment on compute une loss pour un rnn.

In [ ]:
#train the agent
dataset=replay_buffer.as_dataset(num_steps=40) #40 : taille de la séquence, correspond au nb de cellule du rnn, pour compute la loss
for batched_experience in dataset:
    print(batched_experience)
    agent.train(batched_experience)

(Trajectory(
{'action': <tf.Tensor: shape=(40,), dtype=int64, numpy=
array([2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])>,
 'discount': <tf.Tensor: shape=(40,), dtype=float32, numpy=
array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1.], dtype=float32)>,
 'next_step_type': <tf.Tensor: shape=(40,), dtype=int32, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)>,
 'observation': <tf.Tensor: shape=(40, 5), dtype=float64, numpy=
array([[871.17, 871.05, 878.9 , 886.69, 883.48],
       [871.05, 878.9 , 886.69, 883.48, 880.33],
       [878.9 , 886.69, 883.48, 880.33, 882.55],
       [886.69, 883.48, 880.33, 882.55, 879.51],
       [883.48, 880.33, 882.55, 879.51, 883.96],
       [880

/Users/julesripoll/miniforge3/envs/rl/lib/python3.8/site-packages/tf_agents/utils/common.py:1432: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  return state is not None and state is not () and state is not []
/Users/julesripoll/miniforge3/envs/rl/lib/python3.8/site-packages/tf_agents/utils/common.py:1432: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  return state is not None and state is not () and state is not []


TypeError: Input type not supported: (Trajectory(
{'action': <tf.Tensor: shape=(40,), dtype=int64, numpy=
array([2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])>,
 'discount': <tf.Tensor: shape=(40,), dtype=float32, numpy=
array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1.], dtype=float32)>,
 'next_step_type': <tf.Tensor: shape=(40,), dtype=int32, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)>,
 'observation': <tf.Tensor: shape=(40, 5), dtype=float64, numpy=
array([[871.17, 871.05, 878.9 , 886.69, 883.48],
       [871.05, 878.9 , 886.69, 883.48, 880.33],
       [878.9 , 886.69, 883.48, 880.33, 882.55],
       [886.69, 883.48, 880.33, 882.55, 879.51],
       [883.48, 880.33,